### Método de Newton para Sistemas No Lineales
---
**Sistema de ecuaciones:**
- x₁² + 4x₂² = 4
- 4x₁² + x₂² = 4

**Método:** Resolver usando Newton con Gauss-Jordan (sin usar matriz inversa)


In [1]:
# Gauss clase anterior 
class GaussSolution:
    def _print_matrix(self, matrix):
        for row in matrix:
            print([round(x, 6) if abs(x) > 1e-10 else 0 for x in row])
    
    def _create_augmented_matrix(self, coefficients, constants):
        """
        Crea una matriz aumentada a partir de la matriz de coeficientes y el vector de constantes
        
        Args:
            coefficients (list): Matriz de coeficientes (m x n)
            constants (list): Vector de constantes (m x 1)
            
        Returns:
            list: Matriz aumentada (m x (n+1))
        """
        augmented = []
        for i in range(len(coefficients)):
            augmented.append(coefficients[i].copy())
            augmented[i].append(constants[i])
        return augmented
    
    def _extract_solution(self, augmented, num_vars):
        solution = [0] * num_vars
        for i in range(num_vars):
            if i < len(augmented):
                row = augmented[i]
                non_zero = sum(1 for j in range(num_vars) if abs(row[j]) > 1e-10)
                
                if non_zero == 1:
                    for j in range(num_vars):
                        if abs(row[j]) > 1e-10:
                            solution[j] = row[-1] / row[j]
                            break
                else:
                    return None
            else:
                return None
                
        return solution
    
    def gauss_jordan(self, coefficients, constants):
        """
        Resuelve un sistema de ecuaciones lineales usando eliminación Gauss-Jordan sin pivoteo
        
        Args:
            coefficients (list): Matriz de coeficientes (m x n)
            constants (list): Vector de constantes (m x 1)
            
        Returns:
            list: Vector solución o None si no existe una solución única
        """
        # Crear matriz aumentada [A|b]
        augmented = self._create_augmented_matrix(coefficients, constants)
        m = len(augmented) 
        n = len(augmented[0]) - 1 
        
        # Eliminación hacia adelante
        for i in range(min(m, n)):
            if abs(augmented[i][i]) < 1e-10:
                print(f"Error: Pivote cero encontrado en la fila {i}. Intenta usar pivoteo parcial.")
                return None
            
            pivot = augmented[i][i]
            for j in range(i, n + 1):
                augmented[i][j] /= pivot
            
            # Eliminar otras filas
            for k in range(m):
                if k != i:
                    factor = augmented[k][i]
                    for j in range(i, n + 1):
                        augmented[k][j] -= factor * augmented[i][j]
        
        return self._extract_solution(augmented, n)


In [ ]:
# Clase para resolver sistemas no lineales usando el Método de Newton
class NewtonMethod:
    def __init__(self):
        self.gauss_solver = GaussSolution()
    
    def F(self, x):
        """
        Sistema de ecuaciones no lineales:
        f1(x1, x2) = x1^2 + 4*x2^2 - 4
        f2(x1, x2) = 4*x1^2 + x2^2 - 4
        
        Args:
            x (list): Vector [x1, x2]
            
        Returns:
            list: Vector [f1, f2]
        """
        x1, x2 = x[0], x[1]
        f1 = x1**2 + 4*x2**2 - 4
        f2 = 4*x1**2 + x2**2 - 4
        return [f1, f2]
    
    def jacobian(self, x):
        """
        Matriz Jacobiana del sistema:
        J = [∂f1/∂x1  ∂f1/∂x2]   [2*x1   8*x2]
            [∂f2/∂x1  ∂f2/∂x2] = [8*x1   2*x2]
        
        Args:
            x (list): Vector [x1, x2]
            
        Returns:
            list: Matriz Jacobiana 2x2
        """
        x1, x2 = x[0], x[1]
        return [
            [2*x1, 8*x2],
            [8*x1, 2*x2]
        ]
    
    def norm(self, vector):
        """
        Calcula la norma euclidiana de un vector
        
        Args:
            vector (list): Vector
            
        Returns:
            float: Norma del vector
        """
        return sum(v**2 for v in vector)**0.5
    
    def solve(self, x0, tol=1e-8, max_iter=100):
        x = x0.copy()
    
        
        for k in range(max_iter):
            # Calcular F(X^(k-1))
            F_x = self.F(x)
            norm_F = self.norm(F_x)
            
            # Verificar si ya estamos en la solución
            if norm_F < tol:
                return {
                    "solution": x,
                    "iterations": k,
                    "error": norm_F,
                    "converged": True
                }
            
            # Calcular Jacobiana J(X^(k-1))
            J = self.jacobian(x)
            
            # Calcular -F(X^(k-1))
            neg_F = [-F_x[0], -F_x[1]]
            
            
            # J(X^(k-1)) * Y^(k-1) = -F(X^(k-1))
            Y = self.gauss_solver.gauss_jordan(J, neg_F)
            
            if Y is None:
                return {
                    "solution": x,
                    "iterations": k,
                    "error": norm_F,
                    "converged": False
                }
            
            norm_Y = self.norm(Y)
            
            # Actualizar: X^(k) = X^(k-1) + Y^(k-1)
            x = [x[i] + Y[i] for i in range(len(x))]
            
            # Criterio de parada alternativo: si Y es muy pequeño
            if norm_Y < tol:
                F_x = self.F(x)
                norm_F = self.norm(F_x)
                print(f"\n¡Convergencia alcanzada! (||Y|| < tol)")
                print(f"Solución: x1 = {x[0]:.10f}, x2 = {x[1]:.10f}")
                print(f"||F(X)|| = {norm_F:.2e}")
                return {
                    "solution": x,
                    "iterations": k + 1,
                    "error": norm_F,
                    "converged": True
                }
        
        # No convergió en max_iter iteraciones
        F_x = self.F(x)
        norm_F = self.norm(F_x)
    
        print(f"\nAdvertencia: No se alcanzó convergencia en {max_iter} iteraciones")
        print(f"Última aproximación: x1 = {x[0]:.10f}, x2 = {x[1]:.10f}")
        print(f"||F(X)|| = {norm_F:.2e}")
        
        return {
            "solution": x,
            "iterations": max_iter,
            "error": norm_F,
            "converged": False
        }


In [6]:
# Crear instancia del solucionador de Newton
newton = NewtonMethod()

# Prueba 1: Punto inicial (1.0, 1.0)
print("=" * 70)
print("PRUEBA 1: Punto inicial x0 = [1.0, 1.0]")
print("=" * 70)
result1 = newton.solve(x0=[1.0, 1.0], tol=1e-8, max_iter=50)
result1


PRUEBA 1: Punto inicial x0 = [1.0, 1.0]


{'solution': [0.8944271911663216, 0.8944271911663216],
 'iterations': 3,
 'error': 2.104884791220973e-09,
 'converged': True}

In [7]:
# Prueba 2: Punto inicial (2.0, 0.5)
print("\n" + "=" * 70)
print("PRUEBA 2: Punto inicial x0 = [2.0, 0.5]")
print("=" * 70)
result2 = newton.solve(x0=[2.0, 0.5], tol=1e-8, max_iter=50)
result2


PRUEBA 2: Punto inicial x0 = [2.0, 0.5]


{'solution': [0.8944271909999912, 0.8944271909999159],
 'iterations': 5,
 'error': 5.557705048147148e-13,
 'converged': True}

In [8]:
# Prueba 3: Punto inicial (-1.0, -1.0)
print("\n" + "=" * 70)
print("PRUEBA 3: Punto inicial x0 = [-1.0, -1.0]")
print("=" * 70)
result3 = newton.solve(x0=[-1.0, -1.0], tol=1e-8, max_iter=50)
result3



PRUEBA 3: Punto inicial x0 = [-1.0, -1.0]


{'solution': [-0.8944271911663216, -0.8944271911663216],
 'iterations': 3,
 'error': 2.104884791220973e-09,
 'converged': True}